In [2]:
import numpy as np
from keras.layers import Dense, Activation
from keras.layers.recurrent import SimpleRNN, LSTM, GRU
from keras.models import Sequential


# построчное чтение из примера с текстом 
with open("alice_in_wonderland.txt", 'rb') as _in:
    lines = []
    for line in _in:
        line = line.strip().lower().decode("ascii", "ignore")
        if len(line) == 0:
            continue
        lines.append(line)
text = " ".join(lines)
chars = set([c for c in text])
nb_chars = len(chars)


# создание индекса символов и reverse mapping чтобы передвигаться между значениями numerical
# ID and a specific character. The numerical ID will correspond to a column
# ID и определенный символ. Numerical ID будет соответсвовать колонке
# число при использовании one-hot кодировки для представление входов символов
char2index = {c: i for i, c in enumerate(chars)}
index2char = {i: c for i, c in enumerate(chars)}

# для удобства выберете фиксированную длину последовательность 10 символов 
# SEQLEN, STEP = 10, 1 прогон 1 и 2
SEQLEN, STEP = 25, 1 # прогон 3
input_chars, label_chars = [], []

# конвертация data в серии разных SEQLEN-length субпоследовательностей
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i: i + SEQLEN])
    label_chars.append(text[i + SEQLEN])


# Вычисление one-hot encoding входных последовательностей X и следующего символа (the label) y

X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1


# установка ряда метапамертров  для нейронной сети и процесса тренировки
#BATCH_SIZE, HIDDEN_SIZE = 128, 128 - прогон 1 загрузка процессора 60%
#BATCH_SIZE, HIDDEN_SIZE = 256, 256 # прогон 2 загрузка процессора 80%
BATCH_SIZE, HIDDEN_SIZE = 128, 128 # прогон 3 
NUM_ITERATIONS = 150
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100


# Create a super simple recurrent neural network. There is one recurrent
# layer that produces an embedding of size HIDDEN_SIZE from the one-hot
# encoded input layer. This is followed by a Dense fully-connected layer
# across the set of possible next characters, which is converted to a
# probability score via a standard softmax activation with a multi-class
# cross-entropy loss function linking the prediction to the one-hot
# encoding character label.

'''
Создание очень простой рекуррентной нейронной сети. В ней будет один реккурентный закодированный входной слой. За ним последует полносвязный слой связанный с набором возможных следующих символов, которые конвертированы в вероятностные результаты через стандартную softmax активацию с multi-class cross-encoding loss функцию ссылающуются на предсказание one-hot encoding лейбл символа
'''

model = Sequential()
model.add(
    GRU(  # вы можете изменить эту часть на LSTM или SimpleRNN, чтобы попробовать альтернативы
        HIDDEN_SIZE,
        return_sequences=False,
        input_shape=(SEQLEN, nb_chars),
        unroll=True
    )
)
model.add(Dense(nb_chars))
model.add(Activation("softmax"))
model.compile(loss="categorical_crossentropy", optimizer="rmsprop")


# выполнение серий тренировочных и демонстрационных итераций 
for iteration in range(NUM_ITERATIONS):

    # для каждой итерации запуск передачи данных в модель 
    print("=" * 50)
    print("Итерация #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)

    # Select a random example input sequence.
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]

    # для числа шагов предсказаний использование текущей тренируемой модели 
    # конструирование one-hot encoding для тестирования input и добавление предсказания.
    print("Генерация из посева: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):

        # здесь one-hot encoding.
        X_test = np.zeros((1, SEQLEN, nb_chars))
        for j, ch in enumerate(test_chars):
            X_test[0, j, char2index[ch]] = 1

        # осуществление предсказания с помощью текущей модели.
        pred = model.predict(X_test, verbose=0)[0]
        y_pred = index2char[np.argmax(pred)]

        # вывод предсказания добавленного к тестовому примеру 
        print(y_pred, end="")

        # инкрементация тестового примера содержащего предсказание
        test_chars = test_chars[1:] + y_pred
print()


Итерация #: 0
Epoch 1/1
158758/158758 [==============================] - 90s 566us/step - loss: 2.2956
Генерация из посева: e to everything that alic
e to everything that alice the has she was she was she was she was she was she was she was she was she was she was she was sh==================================================
Итерация #: 1
Epoch 1/1
158758/158758 [==============================] - 85s 535us/step - loss: 1.9045
Генерация из посева:  go and take it away! the
 go and take it away! the rast of the reat of the reat of the reat of the reat of the reat of the reat of the reat of the rea==================================================
Итерация #: 2
Epoch 1/1
158758/158758 [==============================] - 88s 552us/step - loss: 1.7496
Генерация из посева: k me for asking! no, itll
k me for asking! no, itll and the was the mad the mad the mad the mad the mad the mad the mad the mad the mad the mad the mad==================================================
Итерация #: 3
Epoch 1/

158758/158758 [==============================] - 85s 535us/step - loss: 1.0896
Генерация из посева: d a fee for obtaining a c
d a fee for obtaining a copyright as the mock turtle in the seamed it as she could not the things in the sea-- much said alic==================================================
Итерация #: 26
Epoch 1/1
158758/158758 [==============================] - 85s 535us/step - loss: 1.0809
Генерация из посева: nd tried to beat them off
nd tried to beat them offed at the project gutenberg-tm electronic works edget herself, and she was a little grow with a litt==================================================
Итерация #: 27
Epoch 1/1
158758/158758 [==============================] - 85s 537us/step - loss: 1.0716
Генерация из посева: e she was talking. how ca
e she was talking. how can in the little good astence its all with the other side to the gryphon said to herself to see it wa==================================================
Итерация #: 28
Epoch 1/1
158758/158758 [====

158758/158758 [==============================] - 88s 555us/step - loss: 0.9551
Генерация из посева:  is! no, indeed, said ali
 is! no, indeed, said alice to herself how she was have the dormouse said in a low, the mouse with the court, and the moral o==================================================
Итерация #: 51
Epoch 1/1
158758/158758 [==============================] - 88s 554us/step - loss: 0.9524
Генерация из посева:  i could not swim-- you c
 i could not swim-- you cant tell you more tonget remave the little golden key in the sea. the hatter was in a more to go to ==================================================
Итерация #: 52
Epoch 1/1
158758/158758 [==============================] - 95s 598us/step - loss: 0.9492
Генерация из посева: of anyone anywhere at no 
of anyone anywhere at no could she thought it such a curious from her into the gryphon, they were goneg too  its head possoni==================================================
Итерация #: 53
Epoch 1/1
158758/158758 [====

158758/158758 [==============================] - 110s 691us/step - loss: 0.9045
Генерация из посева: he happy summer days. the
he happy summer days. there was a large cat only said this, she was so much said, said the gryphon, and the gryphon said all ==================================================
Итерация #: 76
Epoch 1/1
158758/158758 [==============================] - 108s 683us/step - loss: 0.9019
Генерация из посева: rried out of the room. th
rried out of the room. the hatter was not more, said the cat. why, what to do nerer the sender tone, and the mock turtle into==================================================
Итерация #: 77
Epoch 1/1
158758/158758 [==============================] - 97s 612us/step - loss: 0.9020
Генерация из посева: afterwards, it occurred t
afterwards, it occurred to herself, in the project gutenberg-tm clacesiss them the processar. the cat all looking at the play==================================================
Итерация #: 78
Epoch 1/1
158758/158758 [==

158758/158758 [==============================] - 83s 521us/step - loss: 0.8788
Генерация из посева: he whole head appeared, a
he whole head appeared, and the moral of the sentann in a long as it was very suchestang to grow up as the mock turtle indigg==================================================
Итерация #: 101
Epoch 1/1
158758/158758 [==============================] - 83s 524us/step - loss: 0.8776
Генерация из посева: carroll *** end of this p
carroll *** end of this project gutenberg-tm encass, and remement the sound as the sount of little ching have the duchess whe==================================================
Итерация #: 102
Epoch 1/1
158758/158758 [==============================] - 83s 523us/step - loss: 0.8757
Генерация из посева: e caterpillar. well, i sh
e caterpillar. well, i shouldnt gree with a fouthing to the part of things of this agreement she came up to the shriek of the==================================================
Итерация #: 103
Epoch 1/1
158758/158758 [=

158758/158758 [==============================] - 83s 525us/step - loss: 0.8615
Генерация из посева: , copying or distributing
, copying or distributing the soldiers crowded round her hand, and made a minute or two, and alone, he was going off any more==================================================
Итерация #: 126
Epoch 1/1
158758/158758 [==============================] - 87s 547us/step - loss: 0.8621
Генерация из посева: erly the lobsters and the
erly the lobsters and the tail, and the mouse with the door find him redifitides and the tremplang her own course, they were ==================================================
Итерация #: 127
Epoch 1/1
158758/158758 [==============================] - 91s 576us/step - loss: 0.8602
Генерация из посева:  it was all very well to 
 it was all very well to her face into its going to dear tail, and then a pair, and then alice could not see it to her formac==================================================
Итерация #: 128
Epoch 1/1
158758/158758 [=